In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pls

%matplotlib inline

In [2]:
event_df = pd.read_csv('event_df.csv',parse_dates=['start_local_dt','end_local_dt','created_dt','published_dt'],low_memory=False)
event_df.head()

,Unnamed: 0,category_id,city,cost_0,cost_1,cost_2,country,created_dt,currency,donation_0,...,tax_75,tax_76,tax_77,tax_78,tax_79,tax_80,tax_81,tax_82,tax_83,tax_84
0,0,111.0,Wauconda,30.0,30.0,30.0,US,2018-06-07 13:32:09+00:00,USD,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,119.0,Elgin,15.0,NaN,NaN,US,2018-07-28 19:47:56+00:00,USD,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,110.0,Olalla,135.0,NaN,NaN,US,2018-06-28 20:37:37+00:00,USD,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,113.0,Waco,20.0,NaN,NaN,US,2018-06-22 11:57:43+00:00,USD,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,105.0,Oakland,18.0,25.0,23.0,US,2018-08-05 23:27:51+00:00,USD,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
event_df.cost_1.notnull().sum()

5310

In [4]:
event_df.loc[event_df.cost_2.notnull(),'cost_2']

0        30.00
4        23.00
5        35.00
8       351.75
13       60.00
         ...  
9106      0.00
9108    100.00
9110      0.00
9111    500.00
9118      0.00
Name: cost_2, Length: 3369, dtype: float64

In [5]:
event_df.id.tail()

9116    50644365661
9117    51730133221
9118    47992205980
9119    51655511024
9120    48041856486
Name: id, dtype: int64

In [6]:
event_df.loc[event_df.cost_1.notnull() == event_df.cost_2.notnull(),['cost_1','cost_2']]

,cost_1,cost_2
0,30.00,30.00
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,25.00,23.00
5,15.00,35.00
6,NaN,NaN
7,NaN,NaN
8,351.75,351.75
9,NaN,NaN


In [7]:
print(event_df.cost_0.count())
print(event_df.fee_0.notnull().sum())
print(event_df.tax_0.notnull().sum())
print(event_df.free_0.notnull().sum())

9121
9121
9121
9121


In [8]:
event_df.loc[((event_df.cost_0 == 0) & (event_df.cost_1.isnull()))
                      | ((event_df.cost_0 == event_df.cost_1)),
                        ['cost_0','cost_1']].shape


(1640, 2)

Delete the ticket_class expansions beyond index 0

In [9]:
filter_col = [col for col in event_df if col.startswith(('cost_','fee_','tax_','donation_','free_','maximum_quantity_','on_sale_status_','include_fee_'))]

In [10]:
unwanted_list = ['cost_0','fee_0','tax_0','donation_0','free_0','maximum_quantity_0','on_sale_status_0','include_fee_0']
filter_col = [ele for ele in filter_col if ele not in unwanted_list]

In [11]:
event_df.drop(filter_col,axis=1,inplace=True)

In [12]:
event_df.shape

(9121, 38)

In [13]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9121 entries, 0 to 9120
Data columns (total 38 columns):
Unnamed: 0                 9121 non-null int64
category_id                8410 non-null float64
city                       8891 non-null object
cost_0                     9121 non-null float64
country                    8959 non-null object
created_dt                 9121 non-null datetime64[ns, UTC]
currency                   9121 non-null object
donation_0                 9121 non-null float64
end_local_dt               9121 non-null datetime64[ns]
fee_0                      9121 non-null float64
format_id                  8526 non-null float64
free_0                     9121 non-null float64
has_available_tickets      9121 non-null float64
id                         9121 non-null int64
include_fee_0              9121 non-null float64
inventory_type             9121 non-null object
is_paid                    9121 non-null float64
is_reserved_seating        9121 non-null float64


Refund_retention_policy and maximum_quantity_0 have lots of missing values. We will drop these columns

In [14]:
event_df.drop(['refund_retention_policy','maximum_quantity_0'],axis=1,inplace=True)
event_df.shape

(9121, 36)

In [15]:
null_columns = event_df.columns[event_df.isnull().any()]
event_df[null_columns].isnull().sum()

category_id        711
city               230
country            162
format_id          595
postal_code        335
region             322
subcategory_id    3015
venue_id           102
dtype: int64

147 events have no venue expansion and 102 events that had no venue_id. Many of them were online events, so we will identify those and give a venue_id of -999 and change all the address data to 'online'. Those without expansions but with venue ids will be dropped because we will have no idea where they are.

In [16]:
print(event_df.loc[event_df.city.isnull() 
             & event_df.country.isnull() 
             & event_df.postal_code.isnull() 
             & event_df.region.isnull() 
             & event_df.venue_id.isnull(),'venue_id'].shape)

print(event_df.loc[event_df.city.isnull() 
             & event_df.country.isnull() 
             & event_df.postal_code.isnull() 
             & event_df.region.isnull(),'venue_id'].shape)

(102,)
(147,)


In [17]:
event_df.loc[event_df.city.isnull() 
             & event_df.country.isnull() 
             & event_df.postal_code.isnull() 
             & event_df.region.isnull() 
             & event_df.venue_id.isnull(),'venue_id'] = -999
event_df.loc[event_df.venue_id == -999,['city','country','postal_code','region']] = 'online'

There are 147 events without any venue location information. 102 of them are NaN so we can put them into one bin (perhaps online).

In [18]:
null_columns = event_df.columns[event_df.isnull().any()]
event_df[null_columns].isnull().sum()

category_id        711
city               128
country             60
format_id          595
postal_code        233
region             220
subcategory_id    3015
dtype: int64

Anything else that is missing in the venue expansion let's just fully drop for now. This amounts to 388 deleted rows. NBD

In [19]:
print(event_df.shape)
print(event_df.loc[event_df.city.isnull() 
             | event_df.country.isnull() 
             | event_df.postal_code.isnull() 
             | event_df.region.isnull() 
             | event_df.venue_id.isnull(),'venue_id'].shape)
event_df = event_df.drop(event_df.loc[event_df.city.isnull() 
             | event_df.country.isnull() 
             | event_df.postal_code.isnull() 
             | event_df.region.isnull() 
             | event_df.venue_id.isnull(),'venue_id'].index,axis=0)
print(event_df.shape)

(9121, 36)
(388,)
(8733, 36)


In [20]:
null_columns = event_df.columns[event_df.isnull().any()]
event_df[null_columns].isnull().sum()

category_id        670
format_id          570
subcategory_id    2848
dtype: int64

In [21]:
event_df.loc[event_df.category_id.isnull() & event_df.subcategory_id.isnull()].shape

(670, 36)

If organizers did not put a category id in, then they also did not put in a sub category id

In [22]:
event_df.loc[event_df.category_id.isnull() & event_df.format_id.isnull()].shape

(437, 36)

For now, let's keep all the null category, subcategory, and format ids, and set them to their own value: -999

In [23]:
event_df.loc[event_df.category_id.isnull(), ['category_id']] = -999
event_df.loc[event_df.subcategory_id.isnull(), ['subcategory_id']] = -999
event_df.loc[event_df.format_id.isnull(), ['format_id']] = -999

In [24]:
null_columns = event_df.columns[event_df.isnull().any()]
event_df[null_columns].isnull().sum()

Series([], dtype: float64)

No more null values!!

In [25]:
event_df.drop('Unnamed: 0',axis=1,inplace=True)
event_df.columns

Index(['category_id', 'city', 'cost_0', 'country', 'created_dt', 'currency',
       'donation_0', 'end_local_dt', 'fee_0', 'format_id', 'free_0',
       'has_available_tickets', 'id', 'include_fee_0', 'inventory_type',
       'is_paid', 'is_reserved_seating', 'is_series', 'is_series_parent',
       'is_sold_out', 'listed', 'locale', 'on_sale_status_0', 'online_event',
       'postal_code', 'published_dt', 'refund_policy', 'region', 'shareable',
       'start_local_dt', 'start_tz', 'subcategory_id', 'tax_0', 'venue_id',
       'waitlist_available'],
      dtype='object')

In [26]:
event_df.city.value_counts()>100

New York          True
Los Angeles       True
Chicago           True
San Francisco     True
Washington        True
                 ...  
Royal Oak        False
Flourtown        False
Sharston         False
Hunt Valley      False
Inwood           False
Name: city, Length: 1933, dtype: bool

In [27]:
categorical_series = event_df.category_id.value_counts()
categorical_series[categorical_series>50].index

Float64Index([ 101.0,  103.0,  110.0,  111.0, -999.0,  105.0,  113.0,  107.0,
               199.0,  102.0,  115.0,  108.0,  104.0,  114.0,  116.0,  106.0,
               112.0,  119.0,  117.0,  118.0],
             dtype='float64')

In [28]:
categorical_series = event_df.groupby('category_id').count()
categorical_series

,city,cost_0,country,created_dt,currency,donation_0,end_local_dt,fee_0,format_id,free_0,...,published_dt,refund_policy,region,shareable,start_local_dt,start_tz,subcategory_id,tax_0,venue_id,waitlist_available
category_id,,,,,,,,,,,,,,,,,,,,,
-999.0,670,670,670,670,670,670,670,670,670,670,...,670,670,670,670,670,670,670,670,670,670
101.0,1306,1306,1306,1306,1306,1306,1306,1306,1306,1306,...,1306,1306,1306,1306,1306,1306,1306,1306,1306,1306
102.0,308,308,308,308,308,308,308,308,308,308,...,308,308,308,308,308,308,308,308,308,308
103.0,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114,...,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114
104.0,260,260,260,260,260,260,260,260,260,260,...,260,260,260,260,260,260,260,260,260,260
105.0,513,513,513,513,513,513,513,513,513,513,...,513,513,513,513,513,513,513,513,513,513
106.0,174,174,174,174,174,174,174,174,174,174,...,174,174,174,174,174,174,174,174,174,174
107.0,416,416,416,416,416,416,416,416,416,416,...,416,416,416,416,416,416,416,416,416,416
108.0,281,281,281,281,281,281,281,281,281,281,...,281,281,281,281,281,281,281,281,281,281


Get rid of any categorical data that has less than 20 items in it.

In [29]:
event_df = event_df.drop(['city','postal_code'],axis=1)

In [30]:
categorical_cols = ['category_id', 'country', 'currency',
       'donation_0', 'format_id', 'free_0',
       'has_available_tickets', 'include_fee_0', 'inventory_type',
       'is_paid', 'is_reserved_seating',
       'is_sold_out', 'locale', 'on_sale_status_0', 'online_event',
       'refund_policy', 'region', 'shareable',
       'start_tz', 'subcategory_id',
       'waitlist_available']

In [31]:
print(event_df.shape)
for cat in categorical_cols:
#     print(event_df.shape)
    cat_counts = event_df[cat].value_counts()
#     print(cat_counts)
    high_cat_counts = list(cat_counts[cat_counts>20].index)
#     print(high_cat_counts)
    print(cat)
    event_df = event_df.drop(event_df.loc[~event_df[cat].isin(high_cat_counts)].index,axis=0)
    print(event_df.shape)
print(event_df.shape)


(8733, 33)
category_id
(8733, 33)
country
(8630, 33)
currency
(8623, 33)
donation_0
(8623, 33)
format_id
(8623, 33)
free_0
(8623, 33)
has_available_tickets
(8623, 33)
include_fee_0
(8623, 33)
inventory_type
(8623, 33)
is_paid
(8623, 33)
is_reserved_seating
(8623, 33)
is_sold_out
(8623, 33)
locale
(8603, 33)
on_sale_status_0
(8603, 33)
online_event
(8603, 33)
refund_policy
(8603, 33)
region
(8172, 33)
shareable
(8172, 33)
start_tz
(8144, 33)
subcategory_id
(7415, 33)
waitlist_available
(7415, 33)
(7415, 33)
